In [89]:
import pandas as pd
import numpy as np

In [90]:
soil_data = pd.read_csv("soil_data.csv")

In [91]:
soil_data

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
1,1005,31.870670,-85.405456,146,0.0158,0.1868,0.5441,0.2424,0.0106,0.0003,...,72.578804,1.828159,74.406960,3,2,1,1,1,1,1
2,1003,30.659218,-87.746067,52,0.0746,0.4370,0.4415,0.0469,0.0000,0.0000,...,59.843639,2.996914,62.840553,3,2,1,2,1,1,1
3,1007,33.015893,-87.127148,93,0.0144,0.1617,0.3714,0.3493,0.0898,0.0134,...,1.916593,0.008330,1.924924,3,2,1,1,1,1,1
4,1009,33.977448,-86.567246,198,0.0050,0.0872,0.2799,0.3576,0.1477,0.1037,...,1.891909,0.027488,1.919397,3,2,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3104,56037,41.660339,-108.875676,2085,0.0016,0.0281,0.1763,0.3554,0.2121,0.2097,...,0.000000,0.000000,0.000000,1,1,3,1,1,1,3
3105,56039,44.049321,-110.588102,2564,0.0003,0.0026,0.0166,0.0722,0.1489,0.5005,...,2.922309,0.000000,2.922309,1,1,1,1,1,1,1
3106,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
3107,56041,41.284726,-110.558947,2327,0.0050,0.2009,0.4063,0.1858,0.0964,0.1031,...,1.013702,10.755590,11.769293,1,1,2,1,1,1,2


In [92]:
soil_data['drought_index'] = (
    (soil_data['WAT_LAND'].max() - soil_data['WAT_LAND']) +  # Low water content
    (soil_data['NVG_LAND'].max() - soil_data['NVG_LAND']) +  # Low vegetation
    soil_data['URB_LAND']                                    # Urban areas worsen drought
)

In [93]:
features = [
    'slope1', 'slope2', 'slope3', 'slope4', 'slope5', 'slope6', 'slope7', 'slope8',
    'aspectN', 'aspectE', 'aspectS', 'aspectW', 'elevation',
    'WAT_LAND', 'NVG_LAND', 'URB_LAND', 'GRS_LAND', 'FOR_LAND', 'CULTIR_LAND']

In [94]:
X = soil_data[features]
y = soil_data['drought_index']

In [95]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [96]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
# Step 4: Standardize features (critical for Ridge)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 5: Ridge Regression with GridSearchCV
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],  # Test a wider range
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

grid = GridSearchCV(
    Ridge(),
    param_grid,
    cv=5,              # 5-fold cross-validation
    scoring='r2',      # Optimize for R² score
    n_jobs=-1,         # Use all CPU cores
    verbose=1          # Reduced verbosity (set to 3 for detailed output)
)

# Fit the model
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 49 candidates, totalling 245 fits


GridSearchCV(cv=5, estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'solver': ['auto', 'svd', 'cholesky', 'lsqr',
                                    'sparse_cg', 'sag', 'saga']},
             scoring='r2', verbose=1)

In [99]:
print("Best parameters:", grid.best_params_)
print("Best R² (training):", grid.best_score_)

Best parameters: {'alpha': 0.001, 'solver': 'auto'}
Best R² (training): 0.9999999999995237


In [100]:
from sklearn.metrics import r2_score, mean_absolute_error
y_pred = grid.predict(X_test_scaled)
print("Test R²:", r2_score(y_test, y_pred))
print("Test MAE:", mean_absolute_error(y_test, y_pred))

Test R²: 0.9999999999997644
Test MAE: 4.135462751627146e-06


In [101]:
results = pd.DataFrame(grid.cv_results_)
print(results[['params', 'mean_test_score', 'std_test_score']].sort_values(by='mean_test_score', ascending=False))

                                     params  mean_test_score  std_test_score
0        {'alpha': 0.001, 'solver': 'auto'}         1.000000    4.149591e-14
1         {'alpha': 0.001, 'solver': 'svd'}         1.000000    4.149591e-14
2    {'alpha': 0.001, 'solver': 'cholesky'}         1.000000    4.149591e-14
3        {'alpha': 0.001, 'solver': 'lsqr'}         1.000000    3.294389e-14
8          {'alpha': 0.01, 'solver': 'svd'}         1.000000    4.149824e-12
7         {'alpha': 0.01, 'solver': 'auto'}         1.000000    4.149824e-12
9     {'alpha': 0.01, 'solver': 'cholesky'}         1.000000    4.149824e-12
10        {'alpha': 0.01, 'solver': 'lsqr'}         1.000000    3.269768e-12
4   {'alpha': 0.001, 'solver': 'sparse_cg'}         1.000000    1.786423e-09
11   {'alpha': 0.01, 'solver': 'sparse_cg'}         1.000000    1.813322e-09
5         {'alpha': 0.001, 'solver': 'sag'}         1.000000    1.647848e-09
12         {'alpha': 0.01, 'solver': 'sag'}         1.000000    1.310687e-09

In [102]:
import pickle

In [104]:
# Save the scaler
pickle.dump(scaler, open("scaler.sav", "wb"))
# Load the scaler
loaded_scaler = pickle.load(open("scaler.sav", "rb"))
input_data = [[0.0419, 0.2788, 0.2984, 0.2497, 0.1142, 0.017, 0.0, 0.0, 0.1033, 0.1859, 0.2003, 0.1898, 63, 0.0, 0.0, 1.00370001792908, 8.60685634613037, 88.4700469970703, 0.0274876691401005]]
# Scale the input
input_data_scaled = loaded_scaler.transform(input_data)
result = loaded_model.predict(input_data_scaled)


C:\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [105]:
print(result)

[200.98661723]
